Exercise 4. Sentiment analysis with spaCy.

In [10]:
import pandas as pd

# Load the datasets
data_amazon = pd.read_csv('amazon_cells_labelled.txt', header=None, sep='\t', names=['Review', 'Label'])
data_imdb = pd.read_csv('imdb_labelled.txt', header=None, sep='\t', names=['Review', 'Label'])
data_yelp = pd.read_csv('yelp_labelled.txt', header=None, sep='\t', names=['Review', 'Label'])

# Combine the datasets into a single column
comb_data = pd.concat([data_amazon, data_imdb, data_yelp])

comb_data.head()


,Review,Label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
data_amazon.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [12]:
# Add a 'Company' column to each dataset to identify the source
data_amazon['Company'] = 'Amazon'
data_imdb['Company'] = 'IMDB'
data_yelp['Company'] = 'Yelp'

# Combine the datasets into one, including the company identification
comb_data = pd.concat([data_amazon, data_imdb, data_yelp])

comb_data.tail(20)


,Review,Label,Company
980,I have been to very few places to eat that und...,0,Yelp
981,We started with the tuna sashimi which was bro...,0,Yelp
982,Food was below average.,0,Yelp
983,It sure does beat the nachos at the movies but...,0,Yelp
984,"All in all, Ha Long Bay was a bit of a flop.",0,Yelp
985,The problem I have is that they charge $11.99 ...,0,Yelp
986,Shrimp- When I unwrapped it (I live only 1/2 a...,0,Yelp
987,"It lacked flavor, seemed undercooked, and dry.",0,Yelp
988,It really is impressive that the place hasn't ...,0,Yelp
989,I would avoid this place if you are staying in...,0,Yelp


In [13]:
# Explore the structure of comb_data
comb_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2748 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Review   2748 non-null   object
 1   Label    2748 non-null   int64 
 2   Company  2748 non-null   object
dtypes: int64(1), object(2)
memory usage: 85.9+ KB


In [14]:
# Save the combined dataset to a CSV file named 'Sentiment_Analysis_Dataset.csv'
comb_data.to_csv('Sentiment_Analysis_Dataset.csv', index=False)


In [15]:
# Print the columns of the comb_data dataset
comb_data.columns


Index(['Review', 'Label', 'Company'], dtype='object')

In [16]:
# Check for null values in the comb_data dataset
comb_data.isnull().sum()


Review     0
Label      0
Company    0
dtype: int64

In [17]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords


In [18]:

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

stopwords_list = list(stopwords)



In [19]:
import string
import spacy


In [20]:
punctuations = string.punctuation


In [21]:
nlp = spacy.load('en_core_web_sm')
parser = nlp

In [22]:
import spacy

nlp = spacy.load('en_core_web_sm')


sentence_list = []
for review in comb_data['Review']:
    doc = nlp(review)
    sentences = [sentence.text for sentence in doc.sents]
    sentence_list.append(sentences)



In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC


In [24]:
from sklearn.base import TransformerMixin

class Predictors(TransformerMixin):
    def __init__(self):
        # Initialization code can go here
        pass

    def fit(self, X, y=None, **fit_params):
        # Fit the transformer to the data
        # Since this is just a template, we'll just return self
        return self

    def transform(self, X, **transform_params):
        # Transform the data and return the transformed version
        # For the sake of example, we just return X here
        return X

    def get_params(self, deep=True):
        # Return parameters for this estimator
        return {}


In [25]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

nlp = spacy.load('en_core_web_sm')

def clean_text(text):

    text = text.translate(str.maketrans('', '', string.punctuation))


    text = text.lower()


    text = ' '.join([word for word in text.split() if word not in stopwords])

    #
    doc = nlp(text)
    text = ' '.join([token.lemma_ for token in doc])

    return text


In [26]:
# Applying the clean_text function to each entry in the Review column of comb_data
comb_data['Cleaned_Review'] = comb_data['Review'].apply(clean_text)


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(comb_data['Cleaned_Review'])
y = comb_data['Label']

# Split the dataset into a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the LinearSVC classifier
classifier = LinearSVC()
classifier.fit(X_train, y_train)

#  Make predictions and evaluate the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.81
